# Коротко про NumPy - vol. II

О чём сегодня поговорим:
- агрегации (или что делать, если нам нужны скаляры)
- операции над массивами и broadcasting (или как можно складывать нескладываемое)
- маски и булева логика (или как выбрать только то, что нам нужно)
- структурирвоанные массивы (или преамбула к pandas) 

In [1]:
import numpy as np
np.__version__

'1.25.2'

# Часть 1. Агрегации

До этого мы преимущественно работали с массивами как таковыми. Проводили различные операции над ними, однако порой проще работать со скалярными значениями, которые представляют собой какие-то конкретные атрибуты/характеристики векторного объекта. В этом нам помогут агрегации - процедуры, направленные на объединение элементов.

In [2]:
x = np.arange(1, 11)
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [3]:
print("Сумма элементов массива: ", np.sum(x))

Сумма элементов массива:  55


Кстати, обратите внимание, что лучше использовать функции и NumPy, потому что они оптимизированы под векторные вычисления, о которых мы говорили в прошлом занятии.

In [4]:
big_data = np.random.random(10_000_000)
%timeit sum(big_data)
%timeit np.sum(big_data)

413 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.93 ms ± 6.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Давайте посмотрим, какие функции агрегации доступны в NumPy (обратите внимание, что это не исчерпывающий список, а лишь примеры)

In [5]:
x = np.arange(1, 11)
print("Минимальное  значение: ", np.min(x))
print("Максимальное значение: ", np.max(x))
print("Среднее      значение: ", np.mean(x))
print("Произведение значений: ", np.prod(x))

Минимальное  значение:  1
Максимальное значение:  10
Среднее      значение:  5.5
Произведение значений:  3628800


До текущего момента мы использовали агрегации только по отношению к одномерным массивам, а как обстоят дела с многомерными?

In [6]:
x = np.arange(1, 7).reshape(2, 3)
x

array([[1, 2, 3],
       [4, 5, 6]])

При использовании агрегаций без дополнительных параметров они будут обрабатывать все элементы, как если бы они были одномерным массивом

In [7]:
print("Минимальное  значение: ", np.min(x))
print("Максимальное значение: ", np.max(x))
print("Среднее      значение: ", np.mean(x))
print("Произведение значений: ", np.prod(x))

Минимальное  значение:  1
Максимальное значение:  6
Среднее      значение:  3.5
Произведение значений:  720


Однако в случае нескольких измерений у нас появляется возможность явно указывать измерение, вдоль которого будет производиться агрегация

In [8]:
print("Минимальное   значение (по  строкам): ", np.min(x, axis=0))
print("Минимальное   значение (по столбцам): ", np.min(x, axis=1))
print("Максимальное  значение (по  строкам): ", np.max(x, axis=0))
print("Максимальное  значение (по столбцам): ", np.max(x, axis=1))

Минимальное   значение (по  строкам):  [1 2 3]
Минимальное   значение (по столбцам):  [1 4]
Максимальное  значение (по  строкам):  [4 5 6]
Максимальное  значение (по столбцам):  [3 6]


Иначе говоря: в аргументы `axis` мы указываем изменение, по которому массив будет "схлопнут"

# Часть 2. Операции над массивами и Broadcasting

## 2.1. Что такое Broadcasting

Ранее мы применяли различные универсальные функции к объектам одной размерности (в случае с бинарными операциями), однако на этом их возможности не заканчиваются. Формально broadcast'ингом называется набор процедур, которые позволяет применять UFuncs для массивов различного размера. Посмотрим на практике, как это работает

In [9]:
# освежим немного память
# массивы одной размерности
x = np.array([1, 2, 3])
y = np.array([5, -3, -7])

print("x + y = ", x + y)

x + y =  [ 6 -1 -4]


In [10]:
# а теперь вернёмся к примеру, который уже был ранее,
# но на него не обратили внимание: добавим к массиву скаляр
# (очевидно, что их размеры не совпадают)
print("x + 7 = ", x + 7)

x + 7 =  [ 8  9 10]


In [11]:
# фактически эта операция эквивалентна следующей
print("x + [7 7 7] = ", x + np.array([7, 7, 7]))

x + [7 7 7] =  [ 8  9 10]


В этом и заключается вся "магия" broadcasting'а: NumPy под капотом без явной дупликации (для оптимизации расхода памяти) "растягивает" скаляр до нужных размеров, чтобы сделать операции совместными. Конечно, broadcasting не всегда можно применить (об этом правилах приведения размеров поговорим позже), а пока давайте посмотрим на ещё пару примеров

In [12]:
m = np.ones((3, 3), dtype=int)
m

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [13]:
# приведение одномерного массива к двумерному
m + x

array([[2, 3, 4],
       [2, 3, 4],
       [2, 3, 4]])

In [14]:
z = np.array([2, -4, 1]).reshape(3, 1)
z

array([[ 2],
       [-4],
       [ 1]])

In [15]:
# сложим строчку со столбцом (да, возможно даже такое)
x + z

array([[ 3,  4,  5],
       [-3, -2, -1],
       [ 2,  3,  4]])

## 2.2. Правила приведения размеров массивов

Как было скзаано ранее, broadcasting - это не панацея. Есть определённые правила и ограничения, которые нужно деражть в голове, чтобу получить тот результат, который соответствует цели и ожиданиям.

1) Если два массива отличаются по количеству измерений, то форма массива с меньшим количеством измерений дополняется единицами с ведущей (левой) стороны
2) Если форма двух массивов не совпадает в каком-либо измерении, то массив с формой, равной 1 в этом измерении, растягивается до соответствия другой форме
3) Если в каком-либо измерении размеры не совпадают и ни один из них не равен 1, то выдается ошибка

Применяя описанные выше правила, размеры массивов либо будут приведены к форме, допускающей проведение опреации, либо выдастся сообщение о соответствующей ошибке

## 2.3. Примеры работы

### Пример 1

In [16]:
x = np.ones((2, 3), dtype=int)
y = np.arange(3, dtype=int)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (2, 3)
y.shape:  (3,)


Что будет происходить? Рассмотрим по порядку:

1) размеры массивов не совпадают => согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) мы растягиваем размер первого массива вдоль несогласованного измерения, т.е. `y.shape => (2, 3)`

In [17]:
 # финально
x + y

array([[1, 2, 3],
       [1, 2, 3]])

### Пример 2

In [18]:
x = np.arange(3, dtype=int).reshape((3, 1))
y = np.arange(3, dtype=int)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (3, 1)
y.shape:  (3,)


Вновь обратимся к правилам:

1) согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) ОБА массива растягиваются вдоль тех измерений, где есть различия, т.е. `x.shape => (3, 3)` и `y.shape => (3, 3)`

In [19]:
 # финально
x + y

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Пример 3

In [20]:
x = np.ones((3, 2), dtype=int)
y = np.arange(3)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (3, 2)
y.shape:  (3,)


Смотрим в очередной раз:

1) согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) массив `y` растягивается вдоль тех измерений, где есть различия, т.е. `y.shape => (3, 3)`

На выходе получаем `x.shape = (3, 2)` и `y.shape => (3, 3)`, что даёт нам невозможность провести broadcasting по правилу (3)

In [21]:
try:
    x + y
except Exception as e:
    print("ERROR:\n", e)

ERROR:
 operands could not be broadcast together with shapes (3,2) (3,) 


# Часть 3. Маски и булева логика

## 3.1. Использование операторов сравнения в качестве UFuncs

Ранее в разделе про универсальные функции мы говорили о том, какие они бывают и в целом для чего предназначались. Однако их полезность на преобразовании не заканчиваются. Из курса дискретной математики или логики вы наверняка помните, что значения могут быть истинными (True) или ложными (False). В случае с использованием UFuncs есть специальные процедуры, позволяющие работать с этими значениями

In [22]:
x = np.arange(1, 6)
x

array([1, 2, 3, 4, 5])

In [23]:
# удовлетворяет ли значение условию
x <= 2

array([ True,  True, False, False, False])

In [24]:
# удовлетворяет ли значение условию
x > 3

array([False, False, False,  True,  True])

Таким образом, можно проводить проверки, удовлетворяет ли объект на определённой позиции заданному свойству. Обратите внимание, что для многомерных массивов всё работает по абсолютно тому же принципу

In [25]:
x = np.arange(1, 7).reshape((2, 3))
x

array([[1, 2, 3],
       [4, 5, 6]])

In [26]:
x < 3

array([[ True,  True, False],
       [False, False, False]])

## 3.2. Булевы массивы

Немного углубим наши воспоминания в дискретной математике. Чаще всего истинное значение рассматривалось эквивалентным 1, а ложное - 0. Этим знанием можно пользоваться, чтобы вести подсчёт элементов, удовлетворяющим определённым свойствам

In [27]:
x = np.arange(1, 13).reshape(4, 3)
x

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [28]:
# посчитаем количество значений, меньших чем 5
np.count_nonzero(x < 5)

4

In [29]:
# посчитаем количество значений, меньших чем 5, но уже через сумму
np.sum(x < 5)

4

In [30]:
# если функция np.count_nonzero работает без указания измерения,
# то np.sum может провести агрегацию вдоль указанного измерения
# например, сколько значений меньше 6 в каждой строке
np.sum(x < 6, axis=1)

array([3, 2, 0, 0])

Также порой удобно использовать функции `np.any` и `np.all` для проверки, выполняется ли заданное условие для хотя бы одного или для всех элементов соответственно

In [31]:
# есть ли хотя бы одно число, большее 8
np.any(x > 8)

True

In [32]:
# есть ли хотя бы одно число, меньшее 0
np.any(x < 0)

False

In [33]:
# все ли значения в массиве меньше 100
np.all(x < 100)

True

In [34]:
# все ли значения в массиве больше 3
np.all(x > 3)

False

Обратите внимание, что аналогично `np.sum` функции `np.any` и `np.all` могут быть применены к объектам выбранного измерения

## 3.3. Использование булевых массивов в качестве масок

И в заключение этой части очень важный трюк. В NumPy у нас есть не только возможность проверять каждый элемент на соответствие определённому условию, но и выбирать эти элементы для последующей обработки

In [35]:
x = np.arange(1, 13).reshape(4, 3)
x

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [36]:
# запишем в маску результат проверки (x < 5) | (x > 10)
mask = (x < 5) | (x > 10)
mask

array([[ True,  True,  True],
       [ True, False, False],
       [False, False, False],
       [False,  True,  True]])

In [37]:
# выберем элементы, которые удовлетворяют этому условию,
# т.е. мы выбрали все числа, которые меньше 5 или больше 10
x[mask]

array([ 1,  2,  3,  4, 11, 12])

Таким образом, комбинируя разные условия можно создавать специализирванные подвыборки из массивов

# Часть 4. Структурированные массивы

До этого момента мы работали преимущественно с числовыми векторами. Однако на практике часто возникают задачи, когда нужно работать с массивами каких-либо более сложных сущностей: например, нужно обработать миллион заказов, у каждого из которых есть:
- дата совершения
- сумма заказа
- дата доставки

В таком случае мы уже имеем дело с целыми структурами, обо обработке которых с помощью NumPy мы и поговорим

In [38]:
platforms = ["website", "ios", "android", "windows phone"]
num_active_versions = [14, 4, 8, 1]
complexity = [1.0, 1.4, 0.9, 2.1]

Выше представлены фейковые данные о каком-нибудь приложении, которое представлено на ряде платформ. На каждой из платформ может быть своё число активных версий (в зависимости от процедуры AB-тестирования). Кроме того, каждая из платформа имеет свой порог вхождения в разработку и сложность поддержки.

Конечно, можно обрабатывать каждый из списков отдельно, но это не очень удобно. Посмотрим, что может предложить NumPy

In [39]:
data = np.zeros(
    4,
    dtype={
        "names": ["platform", "num_active_versions", "complexity"],
        "formats": ["U16", "i4", "f8"]
    }
)
print(data.dtype)

[('platform', '<U16'), ('num_active_versions', '<i4'), ('complexity', '<f8')]


Теперь, когда подготовлен некоторый placeholder, мы можем наполнить его реальными данными

In [40]:
data["platform"] = platforms
data["num_active_versions"] = num_active_versions
data["complexity"] = complexity

for _ in data:
    print(_)

('website', 14, 1.)
('ios', 4, 1.4)
('android', 8, 0.9)
('windows phone', 1, 2.1)


Посмотрим, какие возможности у нас теперь есть

In [41]:
# вытаскивать список всех платформ
data["platform"]

array(['website', 'ios', 'android', 'windows phone'], dtype='<U16')

In [42]:
# вытаскивать все атрибуты для конкретной строки
data[3]

('windows phone', 1, 2.1)

In [43]:
# использовать маски для подвыборки
print(data[data["complexity"] > 1.0]["platform"])

['ios' 'windows phone']


На самом деле структурированные массивы являются концепцией, крайне близкой к таблицам pandas, но об этом речь уже пойдёт в следующем занятии (: